# SNP Grid

Access the SNP data stored in [BigQuery](https://cloud.google.com/bigquery/docs/).

## Import the libraries used in this tutorial

In [1]:
from google.cloud import bigquery
import pandas

ModuleNotFoundError: No module named 'google.cloud'

In [29]:
#!gcloud config set account "hongping.liang@jax.org"
project_id = "jax-gedi-sandbox-nc-01"
location = "us-east1"

!gcloud config set project $project_id

Updated property [core/project].


## Initialize a client

In [30]:
client = bigquery.Client()

To explicitly specify a project when constructing the client, set the `project` parameter:

## Run a query to data frame

In [31]:
query = """
    SELECT strain_id, strain_name, ingestion_datetime
      FROM `jax-gedi-sandbox-nc-01.mouse_snp.strain_info`
"""
query_job = client.query(query)  
df = query_job.to_dataframe()
df

ValueError: The pyarrow library is not installed, please install pyarrow to use the to_arrow() function.

In [25]:
query_job

## Run SNP query

In [32]:
# Define the query
query = """
   SELECT chr, start_position, rs, strain_id, strain_name, genotyp,
          imputed_flag, percent_impluted, percent_missing
    FROM `jax-human-t2d-cube-ctl.mouse_snp.chr6_flat` 
    LIMIT 1000
"""
query_job = client.query(query)  
df = query_job.to_dataframe()
df

ValueError: The pyarrow library is not installed, please install pyarrow to use the to_arrow() function.